<a href="https://colab.research.google.com/github/LucienShui/HelloMachineLearning/blob/master/Exercise_Flowers_with_Data_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Packages

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import numpy as np
import glob
import shutil
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Data Loading

In [ ]:
URL = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"

zip_file = tf.keras.utils.get_file(origin=URL,
                                   fname="flower_photos.tgz",
                                   extract=True)

base_dir = os.path.join(os.path.dirname(zip_file), "flower_photos")

In [ ]:
print(base_dir)
print(os.listdir(base_dir))

In [ ]:
classes = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

In [ ]:
for class_name in classes:
    img_path = os.path.join(base_dir, class_name)
    images = glob.glob(img_path + '/*.jpg')
    print("{}: {} Images".format(class_name, len(images)))
    train, val = images[:round(len(images) * 0.8)], images[round(len(images) * 0.8):]

    for image in train:
        if not os.path.exists(os.path.join(base_dir, 'train', class_name)):
            os.makedirs(os.path.join(base_dir, 'train', class_name))
        try:
            shutil.move(image, os.path.join(base_dir, 'train', class_name))
        except Exception as e:
            pass
    
    for image in val:
        if not os.path.exists(os.path.join(base_dir, 'val', class_name)):
            os.makedirs(os.path.join(base_dir, 'val', class_name))
        try:
            shutil.move(image, os.path.join(base_dir, 'val', class_name))
        except Exception as e:
            pass
    
    os.system('rm -r ' + os.path.join(base_dir, class_name))

In [ ]:
total_train = 0
for class_name in classes:
    cnt = len(os.listdir(os.path.join(base_dir, 'train', class_name)))
    print("train/{}: {}".format(class_name, cnt))
    total_train += cnt

print("Total Train Data: {}".format(total_train))

print("========")

total_val = 0
for class_name in classes:
    cnt = len(os.listdir(os.path.join(base_dir, 'val', class_name)))
    print("val/{}: {}".format(class_name, cnt))
    total_val += cnt

print("Total Validation Data: {}".format(total_val))

## Creating Data generator

In [ ]:
BATCH_SIZE = 100
IMG_SHAPE = 128

In [ ]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()

### Creating Training Data generator

In [ ]:
image_generator_train = ImageDataGenerator(
    rescale=1./255, # 重新调整
    rotation_range=45,
    width_shift_range=0.2, # 宽度偏移范围
    height_shift_range=0.2,
    shear_range=0.2, # 剪切范围
    zoom_range=0.2, # 变焦范围
    horizontal_flip=True, # 水平翻转
    fill_mode='nearest'
)

train_data_gen = image_generator_train.flow_from_directory(batch_size=BATCH_SIZE,
                                                           directory=os.path.join(base_dir, 'train') ,
                                                           shuffle=True,
                                                           target_size=(IMG_SHAPE, IMG_SHAPE),
                                                           class_mode='sparse')

In [ ]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

### Creating Validation Data generator

In [ ]:
image_gen_val = ImageDataGenerator(rescale=1./255)

val_data_gen = image_gen_val.flow_from_directory(batch_size=BATCH_SIZE,
                                                 directory=os.path.join(base_dir, 'val'),
                                                 target_size=(IMG_SHAPE, IMG_SHAPE),
                                                 class_mode='sparse')

## Create the CNN

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SHAPE, IMG_SHAPE, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

## Compile the Model

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

## Train the Model

In [ ]:
EPOCHS = 100

history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=int(np.ceil(total_train / float(BATCH_SIZE))),
    epochs=EPOCHS,
    validation_data=val_data_gen,
    validation_steps=int(np.ceil(total_val / float(BATCH_SIZE)))
)

## Visualizing results of the trainig

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.savefig('./foo.png')
plt.show()